# hw2 task2

## Data Preprocessing

In [1]:
import numpy as np
import pandas as pd
import scipy
import datetime
from collections import defaultdict
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('ntucsie-sdml2018-2-1/rating_train.csv', header=0)

In [3]:
dataset.head()

,date,userid,foodid
0,2014-09-15,6,0
1,2014-09-16,6,0
2,2014-09-16,6,1
3,2014-09-16,6,2
4,2014-09-16,6,3


In [4]:
print('unique userid:', len(dataset.userid.unique()))
print('unique foodid:', len(dataset.foodid.unique()))

unique userid: 2608
unique foodid: 5532


In [ ]:
users = dataset.userid.unique()
user_total_cnt = defaultdict(int)

for userid in users:
    data = dataset[dataset['userid'] == userid]
    user_total_cnt[userid] = data.shape[0]

### Rule-based - count frequency

In [ ]:
matrix = dataset.as_matrix()
freq = defaultdict(lambda: defaultdict(int))
user_cnt = defaultdict(int)
user_freq_cnt = defaultdict(int)
user_foods = defaultdict(set)

for row in matrix:
    date, userid, foodid = row[0], row[1], row[2]
    user_cnt[userid] += 1
    if user_cnt[userid] >= user_total_cnt[userid] * 0.85:
        if foodid not in user_foods[userid]:
            user_foods[userid].add(foodid)
        user_freq_cnt[userid] += 1
        freq[userid][foodid] += 1

In [ ]:
matrix = dataset.as_matrix()
matrix = np.flipud(matrix) # From new to old
freq = defaultdict(lambda: defaultdict(int))
user_date_record = defaultdict(lambda: defaultdict(set))
user_cnt = defaultdict(int)
user_freq_cnt = defaultdict(int)
user_foods = defaultdict(set)
user_stop_date = defaultdict(str)

for row in matrix:
    date, userid, foodid = row[0], row[1], row[2]
    user_cnt[userid] += 1
    if foodid not in user_date_record[userid][date]:
        user_date_record[userid][date].add(foodid)
#         if user_cnt[userid] <= user_total_cnt[userid] * 0.13:
        if userid not in user_stop_date or user_stop_date[userid] == date:
            if foodid not in user_foods[userid]:
                user_foods[userid].add(foodid)
            if len(user_foods[userid]) >= 40:
                user_stop_date[userid] = date
            user_freq_cnt[userid] += 1
            freq[userid][foodid] += 1

### Rule-based - count unqiue foods

In [ ]:
matrix = dataset.as_matrix()
matrix = np.flipud(matrix) # From new to old
freq = defaultdict(lambda: defaultdict(int))
user_freq_cnt = defaultdict(int)
user_total_foods = defaultdict(set)
user_foods = defaultdict(set)

for row in matrix:
    date, userid, foodid = row[0], row[1], row[2]
    user_cnt[userid] += 1
    if foodid not in user_total_foods[userid]:
        user_total_foods[userid].add(foodid)
    if len(user_foods[userid]) < 20:
        if foodid not in user_foods[userid]:
            user_foods[userid].add(foodid)
        user_freq_cnt[userid] += 1
        freq[userid][foodid] += 1

### Prediction

In [ ]:
train = []
for userid in freq:
    for foodid in freq[userid]:
        train.append([userid, foodid, freq[userid][foodid]])
train = np.array(train)

df = pd.DataFrame()
df['userid'] = train[:,0]
df['foodid'] = train[:,1]
df['freq'] = train[:,2]

In [ ]:
with open('pred.csv', 'w') as file:
    file.write('userid,foodid\n')
    
    for userid in users:
        if len(user_foods[userid]) < 20:
            print(userid, len(user_foods[userid]))
        foods = df[df['userid'] == userid].sort_values('freq', ascending=False)['foodid'][:20]
        file.write('{},'.format(userid))
        cnt = 0
        for foodid in foods:
            file.write('{}'.format(foodid))
            cnt += 1
            if cnt >= 20:
                file.write('\n')
                break
            else:
                file.write(' ')